# Proceso de Transformacion de columnas 

Desanidar Columna  production_companies y production_countries

* Columna Production_companies

In [1]:
# Empiezo importando las librerias que usar en el projecto

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json


In [2]:
# creo un DataFrame en pandas(pd) a partir de los datos contenidos en movies_dataset.csv
pd.set_option('display.max_colwidth', None) # quiero ver los valores completos.
df = pd.read_csv("/Users/benjaminzelaya/Desktop/carpeta/Proyecto_ML_Individual/ETL - EDA/datasets reestructurados para empezar ETL/data Reestructurados/m_productions.csv")

# Imprimo los primeros registros del DataFrame
df.head()

,production_companies,production_countries,id
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",862
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",8844
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",15602
3,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",31357
4,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",11862


In [3]:
# los valores faltantes (NaN) vamos a convertir convirtiéndolos en una lista vacía
df['production_companies'] = df['production_companies'].fillna('[]')

In [4]:
# Los datos de cadena en listas de diccionarios los vamos a convertir utilizando 'eval()', ya que no puedo desanidar porque estan en string , y no en diccionarios.
import ast 

df['production_companies'] = df['production_companies'].apply(ast.literal_eval)

In [5]:
# utilizo  función explode que me permite  expandir la lista  en filas separadas
df_exploded = df.explode('production_companies', ignore_index=True)


In [6]:
# Extraer los nombres e ids de las compañías en columnas separadas
df_exploded['name_companies'] = df_exploded['production_companies'].apply(lambda x: x.get('name', '') if isinstance(x, dict) else '')
df_exploded['id_companies'] = df_exploded['production_companies'].apply(lambda x: x.get('id', '') if isinstance(x, dict) else '')


In [7]:
# Unimos los valores desanidados  con el DataFrame 
df = df.join(df_exploded[['name_companies', 'id_companies']])

In [8]:
df[['name_companies', 'id_companies']].head()

,name_companies,id_companies
0,Pixar Animation Studios,3
1,TriStar Pictures,559
2,Teitler Film,2550
3,Interscope Communications,10201
4,Warner Bros.,6194


In [9]:
columna_eliminada = ['production_companies','id']
df.drop(columns=columna_eliminada, inplace=True)
# Elimino la columna production_companies y tambien id porque sus datos ya fueron extraidos en columnas diferentes y mas ordenados.


In [10]:
df.head()

,production_countries,name_companies,id_companies
0,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",Pixar Animation Studios,3
1,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",TriStar Pictures,559
2,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",Teitler Film,2550
3,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",Interscope Communications,10201
4,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]",Warner Bros.,6194


* Columna Production_Countries

In [11]:
# los valores faltantes (NaN) vamos a convertir convirtiéndolos en una lista vacía
df['production_countries'] = df['production_countries'].fillna('[]')

In [12]:
# Los datos de cadena en listas de diccionarios los vamos a convertir utilizando 'eval()', ya que no puedo desanidar porque estan en string , y no en diccionarios.
import ast 

df['production_countries'] = df['production_countries'].apply(ast.literal_eval)

In [13]:
# Desanido la columna production_countries y crear nuevas columnas
df_exploded = df.explode('production_countries')

def extract_country_info(x):
    if isinstance(x, dict):
        return pd.Series([x.get('iso_3166_1', ''), x.get('name', '')])
    else:
        return pd.Series(['', ''])

df_exploded[['Cod_countries', 'country_name']] = df_exploded['production_countries'].apply(extract_country_info)

In [14]:
# # Unimos los valores desanidados  con el DataFrame 
df = df.join(df_exploded[['Cod_countries', 'country_name']])

In [15]:
df[['Cod_countries', 'country_name']].head()

,Cod_countries,country_name
0,US,United States of America
1,US,United States of America
2,US,United States of America
3,US,United States of America
4,US,United States of America


In [17]:
columna_eliminada = ['production_countries']
df.drop(columns=columna_eliminada, inplace=True)
# Elimino la columna production_countries porque sus datos ya fueron extraidos en columnas diferentes y mas ordenados.


In [18]:
df.head()

,name_companies,id_companies,Cod_countries,country_name
0,Pixar Animation Studios,3,US,United States of America
1,TriStar Pictures,559,US,United States of America
2,Teitler Film,2550,US,United States of America
3,Interscope Communications,10201,US,United States of America
4,Warner Bros.,6194,US,United States of America


In [19]:
movie_data_1 = df

# Creo el dataframe con las columnas indicadas guardando el mismo dentro la carpeta ETL/EDA del reposotorio
movie_data_1.to_csv("m_productions_modificado.csv", index=False)